In [ ]:
import cv2
import matplotlib.pyplot as plt


In [ ]:
image=cv2.imread("gg.bmp")
o = image.copy()
plt.imshow(image)

In [ ]:
#图像去噪灰度处理
image = cv2.GaussianBlur(image, (3, 3), 0)
plt.imshow(image)

In [ ]:
grayimage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(grayimage)

In [ ]:
import numpy as np

#创建一个6x6的BGR三通道图片
img6x6 = np.zeros((6,6,3), dtype=np.uint8) 
#Blue通道取值0~99随机值
img6x6[:,:,0] = np.random.randint(0, 100, size=36).reshape(6,6)
#Green通道取值100~199随机值
img6x6[:,:,1] = np.random.randint(100, 200, size=36).reshape(6,6)
#Red通道取值200~255随机值
img6x6[:,:,2] = np.random.randint(200, 256, size=36).reshape(6,6)

plt.imshow(img6x6)

In [ ]:
plt.imshow(cv2.threshold(img6x6, 128, 255, cv2.THRESH_BINARY)[1])

In [ ]:
image = cv2.threshold(grayimage, 0, 255, cv2.THRESH_OTSU)[1]
plt.imshow(image)

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
image = cv2.dilate(image, kernel)
plt.imshow(image)

In [ ]:
contours,hierarchy=cv2.findContours(image,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
x = cv2.drawContours(o.copy(), contours, -1, (0, 0, 255), 1)  
plt.imshow(x)

In [ ]:
chars = []
for item in contours:
    rect = cv2.boundingRect(item)  
    x,y,w,h = cv2.boundingRect(item)  
    chars.append(rect)
print("共找到轮廓个数：", len(contours))
print("包围框列表：", chars)

In [ ]:
# 指定的条件参数
aspect_ratio_range = (1.5, 8.0)  # 高宽比范围
min_width = 3  # 最小宽度

# 将包围框按照 x 轴坐标值排序（自左向右排序）
chars = sorted(chars, key=lambda s: s[0], reverse=False)

# 将字符的轮廓筛选出来
plateChars = []
for word in chars:
    if (word[3] > (word[2] * aspect_ratio_range[0])) and (word[3] < (word[2] * aspect_ratio_range[1])) and (word[2] > min_width):
        plateChar = image[word[1]:word[1] + word[3], word[0]:word[0] + word[2]]
        print(plateChar)
        plateChars.append(plateChar)

In [ ]:
for i, im in enumerate(plateChars):
    print("char" + str(i), end=" ")
    plt.subplot(1, len(plateChars), i+1)
    plt.imshow(im, 'gray')
plt.show()

In [ ]:
import glob

templateDict = {
    0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H',
    18: 'J', 19: 'K', 20: 'L', 21: 'M', 22: 'N', 23: 'P', 24: 'Q', 25: 'R',
    26: 'S', 27: 'T', 28: 'U', 29: 'V', 30: 'W', 31: 'X', 32: 'Y', 33: 'Z',
    34: '京', 35: '津', 36: '冀', 37: '晋', 38: '蒙', 39: '辽', 40: '吉', 41: '黑',
    42: '沪', 43: '苏', 44: '浙', 45: '皖', 46: '闽', 47: '赣', 48: '鲁', 49: '豫',
    50: '鄂', 51: '湘', 52: '粤', 53: '桂', 54: '琼', 55: '渝', 56: '川', 57: '贵',
    58: '云', 59: '藏', 60: '陕', 61: '甘', 62: '青', 63: '宁', 64: '新',
    65: '港', 66: '澳', 67: '台'
}

def getcharacters():
    c = []
    for i in range(0, 67):
        words = glob.glob('template/' + templateDict.get(i) + '/*.*')
        c.append(words)
    return c



In [ ]:
def getMatchValue(template, image):
    # 读取模板图像
    templateImage = cv2.imread(template)
    
    # 模板图像色彩空间转换，BGR-->灰度
    templateImage = cv2.cvtColor(templateImage, cv2.COLOR_BGR2GRAY)
    
    # 模板图像阈值处理，灰度-->二值
    ret, templateImage = cv2.threshold(templateImage, 0, 255, cv2.THRESH_OTSU)
    
    # 获取待识别图像的尺寸
    height, width = image.shape
    
    # 将模板图像调整为与待识别图像尺寸一致
    templateImage = cv2.resize(templateImage, (width, height))
    
    # 进行匹配操作并返回匹配值
    matchValue = cv2.matchTemplate(image, templateImage, cv2.TM_CCOEFF_NORMED)
    return matchValue

In [ ]:
def matchChars(plates, chars):
    results = []  # 存储所有的识别结果

    for plateChar in plates:  # 逐个遍历要识别的字符
        bestMatch = []  # 最佳匹配

        for words in chars:  # 遍历字符模板
            match = []  # 每个字符的匹配值

            for word in words:  # 遍历模板
                result = getMatchValue(word, plateChar)  # 计算匹配值
                match.append(result)

            bestMatch.append(max(match, default=0))  # 将每个字符模板的最佳匹配加入bestMatch

        i = bestMatch.index(max(bestMatch, default=0))  # 获取最佳匹配的字符模板索引
        r = templateDict[i]  # 获取单个待识别字符的识别结果
        results.append(r)  # 将每一个分割字符的识别结果加入results列表

    return results  # 返回所有的识别结果
